# Extracting data from NHTSA's APIs.  
[NHTSA Documentation](https://www.nhtsa.gov/nhtsa-datasets-and-apis) 
Purpose: To extract vehicle safety data from NHTSA and store in detla tables for further analysis.
Approach: NHTSA APIs build on each other.  I will follow these steps:
1. Pull the Years that they provide
1. Pull the Make by Year
1. Pull the Models by Year & Make
1. Pull the Trim by Year, Make, & Model
1. Pull the Safety Ratings by Year, Make, Model and Trim 

##Year
**Purpose**: This code block is pulling the years that the NHTSA has data available for.

**Inputs**:
- NHTSA API endpoint: `https://api.nhtsa.gov/SafetyRatings`

**Outputs**:
- A Delta table named `nhtsa_safety_ratings.years`

**Steps**:
1. Import the `requests` library.
1. Define the API endpoint URL.
1. Make a GET request to the NHTSA API.
1. Parse the JSON response.
1. Extract the model years from the response.
1. Convert the list of years to a Spark DataFrame.
1. Create a schema if it doesn't exist.
1. Write the DataFrame to a Delta table.
1. Display the DataFrame.

In [0]:
import requests

base_url = "https://api.nhtsa.gov/SafetyRatings"

response = requests.get(base_url)
data = response.json()

years = [item['ModelYear'] for item in data['Results']]

df_years = spark.createDataFrame(years, "int").toDF("Year")
display(df_years)

spark.sql("CREATE SCHEMA IF NOT EXISTS nhtsa_safety_ratings")
df_years.write.format("delta").mode("overwrite").saveAsTable("nhtsa_safety_ratings.years")

##Make by Year
**Purpose**: This cell fetches the Makes by Year from the NHTSA API and stores the results in a Delta table.

**Inputs**:
- Delta table nhtsa_safety_ratings.years

**Outputs**:
- A Delta table named nhtsa_safety_ratings.make

**Steps**:
1. Define the base URL for the NHTSA API.
1. Collect the model years from the previously created Delta table.
1. Initialize an empty list to store the results.
1. Loop through each year, make a GET request to the NHTSA API, and parse the JSON response.
1. Append the results to the list.
1. Convert the list of results to a Spark DataFrame.
1. Display the DataFrame.
1. Write the DataFrame to a Delta table.

In [0]:
years = [row['Year'] for row in df_years.collect()]

base_url = "https://api.nhtsa.gov/SafetyRatings/modelyear/"

make_results = []
for year in years:
    url = f"{base_url}{year}"
    response = requests.get(url)
    data = response.json()
    for item in data.get('Results', []):
        make_results.append(item)

df_makes = spark.createDataFrame(make_results)
display(df_makes)

df_makes.write.format("delta").mode("overwrite").saveAsTable("nhtsa_safety_ratings.make")

## Model by Year & Make
**Purpose**: This cell fetches the Models by Year and Make from the NHTSA API and stores the results in a Delta table.

**Inputs**:
- Delta table nhtsa_safety_ratings.make 

**Outputs**:
- A Delta table named nhtsa_safety_ratings.model

**Steps**:
1. Read the Delta table into a DataFrame.
1. Collect distinct make and model year combinations.
1. Define the base URL for the NHTSA API.
1. Initialize an empty list to store the results.
1. Loop through each make and model year combination, make a GET request to the NHTSA API, and parse the JSON response.
1. Append the results to the list.
1. Convert the list of results to a Spark DataFrame.
1. Display the DataFrame.
1. Write the DataFrame to a Delta table.

In [0]:
make_year_combinations = df_makes.select("Make", "ModelYear").distinct().collect()

base_url = "https://api.nhtsa.gov/SafetyRatings/modelyear/"

model_results = []
for row in make_year_combinations:
    make = row['Make']
    modelyear = row['ModelYear']
    url = f"{base_url}{modelyear}/make/{make}"
    response = requests.get(url)
    data = response.json()
    for item in data.get('Results', []):
        model_results.append(item)

df_models = spark.createDataFrame(model_results)
display(df_models)

df_models.write.format("delta").mode("overwrite").saveAsTable("nhtsa_safety_ratings.model")

## Variants by Year, Make, & Model
**Purpose**: This cell fetches the Variants (Trim) by Year, Make, and Model from the NHTSA API and stores the results in a Delta table.

**Inputs**:
- Delta table nhtsa_safety_ratings.model

**Outputs**:
- A Delta table named nhtsa_safety_ratings.variant

**Steps**:
1. Read the Delta table into a DataFrame.
1. Collect distinct make and model year combinations.
1. Define the base URL for the NHTSA API.
1. Initialize an empty list to store the results.
1. Loop through each year, make and model, make a GET request to the NHTSA API, and parse the JSON response.
1. Append the results to the list.
1. Convert the list of results to a Spark DataFrame.
1. Display the DataFrame.
1. Write the DataFrame to a Delta table.

In [0]:
# Collect the make and year combinations
model_make_year_combinations = df_models.select("Model","Make", "ModelYear").distinct().collect()

base_url = "https://api.nhtsa.gov/SafetyRatings/modelyear/"

variant_results = []
for row in model_make_year_combinations:
    model = row['Model']
    make = row['Make']
    modelyear = row['ModelYear']
    url = f"{base_url}{modelyear}/make/{make}/model/{model}"
    response = requests.get(url)
    data = response.json()
    for item in data.get('Results', []):
        item['Model'] = model
        item['Make'] = make
        item['ModelYear'] = modelyear
        variant_results.append(item)

df_variants = spark.createDataFrame(variant_results)
display(df_variants)

df_variants.write.format("delta").mode("overwrite").saveAsTable("nhtsa_safety_ratings.variants")


## Safety Ratings by Variants, Year, Make, & Model
**Purpose**: This cell fetches the Safety Ratings by Year, Make, Model, and Variants from the NHTSA API and stores the results in a Delta table.

**Inputs**:
- Delta table nhtsa_safety_ratings.variants

**Outputs**:
- A Delta table named nhtsa_safety_ratings.safety_ratings

**Steps**:
1. Read the Delta table into a DataFrame.
1. Collect distinct make and model year combinations.
1. Define the base URL for the NHTSA API.
1. Initialize an empty list to store the results.
1. Loop through each year, make, model and variant, make a GET request to the NHTSA API, and parse the JSON response.
1. Append the results to the list.
1. Convert the list of results to a Spark DataFrame.
1. Display the DataFrame.
1. Write the DataFrame to a Delta table.

In [0]:
# Collect the make and year combinations
model_make_year_variant_combinations = df_variants.select(
    "VehicleId", "Model", "Make", "ModelYear"
).distinct().collect()

base_url = "https://api.nhtsa.gov/SafetyRatings/"

safety_results = []
for row in model_make_year_variant_combinations:
    vehicleid = row['VehicleId']
    model = row['Model']
    make = row['Make']
    modelyear = row['ModelYear']
    url = f"{base_url}VehicleId/{vehicleid}"
    response = requests.get(url)
    data = response.json()
    for item in data.get('Results', []):
        item['VehicleId'] = vehicleid
        item['Model'] = model
        item['Make'] = make
        item['ModelYear'] = modelyear         
        safety_results.append(item)

df_safety_ratings = spark.createDataFrame(safety_results)
display(df_safety_ratings)

df_safety_ratings.write.format("delta").mode("overwrite").saveAsTable("nhtsa_safety_ratings.safety_ratings")